In [1]:
%pip install seaborn pandas matplotlib
%pip install numpy pandas transformers[sentencepiece] torch torchvision datasets

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [4]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.insert(0, "../")

import numpy as np
import torch
import datasets
import transformers as tfmr
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt

# from backend.logic import parse_accounts, match_cc_mb_entries
from backend.analysis import load_mobills_monthly_categories_df

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
# Load training dataset
dataset_csv = "../data/ml/dataset.csv"
df = pd.read_csv(dataset_csv)

# Remove extra column for indices
df.drop(columns="Unnamed: 0", inplace=True)

# Obtain category labels and ids
category_entries = pd.Categorical(df.category.astype("category"))
categories = list(category_entries.categories)
# Usage: categories.index("Food and Drink") / categories[2]
df["category_id"] = category_entries.codes
# print(f"Categories: {categories}")
ds_orig = datasets.Dataset.from_pandas(df[["name", "category_id"]])
ds_trans_orig = datasets.Dataset.from_pandas(df[["transaction", "category_id"]])

In [7]:
# Tokenize dataset
model_name = "distilbert-base-uncased"  # "distilbert-base-uncased"
tokenizer = tfmr.AutoTokenizer.from_pretrained(model_name)

def tokenize_function(ds):
    return tokenizer(ds["text"], padding="max_length", truncation=True)

#dss = datasets.DatasetDict({"train": ds_orig.rename_column("category_id", "labels").rename_column("name", "text")})
dss = datasets.DatasetDict({"train": ds_trans_orig.rename_column("category_id", "labels").rename_column("transaction", "text")})
tdss = dss.map(tokenize_function, batched=True)
ds_tokenized = tdss["train"]

print("Some samples:")
dss["train"][20:30]

loading configuration file https://huggingface.co/distilbert-base-uncased/resolve/main/config.json from cache at /home/mimic/.cache/huggingface/transformers/23454919702d26495337f3da04d1655c7ee010d5ec9d77bdb9e399e00302c0a1.d423bdf2f58dc8b77d5f5d18028d7ae4a72dcfd8f468e81fe979ada957a8c361
Model config DistilBertConfig {
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.8.1",
  "vocab_size": 30522
}

loading file https://huggingface.co/distilbert-base-uncased/resolve/main/vocab.txt from cache at /home/mimic/.cache/huggingface/transformers/0e1bbfda7f63a99bb52e3915dcf10c3c92122b827d92eb2d34ce94ee79ba486c


Some samples:


{'text': ['KOODO MOBILE PAC EDMONTON AB',
  'COSTCO WHOLESALE W515 MONTREAL QC',
  'COSTCO WHOLESALE W515 MONTREAL QC',
  'BUYAPI BUYAPI 6137930050 ON',
  'LES MARCHES LOUISE MEN MONTRÉAL QC',
  'RAMEN-YA MONTREAL QC',
  'AMZN Mktp CA WWW.AMAZON.CAON',
  'PHO RACHEL MONTREAL QC',
  'PHARMPRIX #0042 MONTREAL QC',
  'STM GUY CONCORDI DIE10 MONTREAL QC'],
 'labels': [1, 4, 4, 7, 4, 4, 7, 4, 4, 10]}

In [6]:
# Finetune pre-trained model

model = tfmr.AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(categories))
training_args = tfmr.TrainingArguments(
    output_dir="/tmp/bookkeeper_bert",
    evaluation_strategy="epoch",
    save_strategy="no",
    num_train_epochs=10,
    learning_rate=5e-5,
)
trainer = tfmr.Trainer(model=model, args=training_args, train_dataset=ds_tokenized, eval_dataset=ds_tokenized)
trainer.train()

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'classifier.bias', 'classifier

Epoch,Training Loss,Validation Loss


The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text.
***** Running Evaluation *****
  Num examples = 1901
  Batch size = 8
The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text.
***** Running Evaluation *****
  Num examples = 1901
  Batch size = 8
The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text.
***** Running Evaluation *****
  Num examples = 1901
  Batch size = 8
The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text.
***** Running Evaluation *****
  Num examples = 1901
  Batch size = 8
The following columns in the evaluation set  don't have a corresponding argument in 

TrainOutput(global_step=2380, training_loss=0.5254615174622095, metrics={'train_runtime': 767.7865, 'train_samples_per_second': 24.759, 'train_steps_per_second': 3.1, 'total_flos': 3910580876359680.0, 'train_loss': 0.5254615174622095, 'epoch': 10.0})

In [9]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text.
***** Running training *****
  Num examples = 1901
  Num Epochs = 10
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 2380


Epoch,Training Loss,Validation Loss


The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text.
***** Running Evaluation *****
  Num examples = 1901
  Batch size = 8
The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text.
***** Running Evaluation *****
  Num examples = 1901
  Batch size = 8
The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text.
***** Running Evaluation *****
  Num examples = 1901
  Batch size = 8
The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text.
***** Running Evaluation *****
  Num examples = 1901
  Batch size = 8
The following columns in the evaluation set  don't have a corresponding argument in 

KeyboardInterrupt: 

In [6]:
# Save model
save_dir = "cc_category_classifier"
tokenizer.save_pretrained(save_dir)
model.save_pretrained(save_dir)

In [7]:
model_name = "cc_category_classifier"
tokenizer = tfmr.AutoTokenizer.from_pretrained(model_name)
model = tfmr.AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(categories))

In [8]:
# Test classifier

model.eval()
item_names = ["PHARMPRIX #0042 MONTREAL QC", "BUYAPI BUYAPI"]
tokenized_input = tokenizer(item_names, padding="max_length", truncation=True, return_tensors="pt").to(model.device)
with torch.no_grad():
    outputs = model(**tokenized_input)

logits = outputs.logits
predictions = torch.argmax(logits, dim=-1)
pred_categories = [categories[pred_id] for pred_id in predictions]

for name, category in zip(item_names, pred_categories):
    print(f"{name} -> {category}")

PHARMPRIX #0042 MONTREAL QC -> Healthcare
BUYAPI BUYAPI -> Hobbies
